In [1]:
from dataclasses import dataclass, field
import os, sys, subprocess
import copy
import statistics
import math
import time
import datetime as dt
import itertools as it
import random
import string
import gzip

import pysam
import numpy as np
import pandas as pd
import pyfaidx
from intervaltree import Interval, IntervalTree

In [2]:
##### Constants #####

LINEAR_COLUMNS = ('read_id', 
				  'chrom', 
				#   'start', 
				#   'end', 
				  'blocks', 
				  'cigar',
				  'read',
				  )
CIGARTUPLE_CODES = {0: 'M',
					1: 'I',
					2: 'D',
					3: 'N',
					4: 'S',
					5: 'H',
					6: 'P',
					7: '=',
					8: 'X',
					9: 'B',
					}

In [3]:
##### Functions #####
def parse_attributes(attribute_string:str, anno_type:str) -> dict:
	if anno_type == 'gtf':
		attributes = attribute_string.rstrip('";').split('; ')
		attributes = [attr.split(' ') for attr in attributes]
		tags = [attr_val.strip('"') for attr_name, attr_val in attributes if attr_name=='tag']
		attributes = {attr_name: attr_val.strip('"') for attr_name, attr_val in attributes if attr_name!='tag'}
		attributes['tags'] = tags
	else:
		attributes = [attr.split('=') for attr in attribute_string.split(';')]
		attributes = [(attr[0].lstrip(), attr[1]) for attr in attributes]
		attributes = dict(attributes)
		if 'tag' in attributes:
			attributes['tags'] = attributes['tag'].split(',')

	return attributes


def parse_gene_info(ref_anno:str) -> dict:
	prev_ext, last_ext = ref_anno.split('.')[-2:]
	if last_ext == 'gz':
		in_file, anno_type = gzip.open(ref_anno, 'rt'), prev_ext
	else:
		in_file, anno_type = open(ref_anno), last_ext

	genes = {}
	for line in in_file:
		if line[0] != '#':
			_, _, feature, _, _, _, _, _, attributes = line.strip().split('\t')
			if feature == 'transcript':
				attributes = parse_attributes(attributes, anno_type)
				genes[attributes['gene_id']] = [attributes[e] for e in ['gene_name', 'gene_type']]
	in_file.close()
	
	return genes


def parse_introns(ref_introns:str) -> tuple:
	'''
	Returns a dict formatted as follows:
	{Chromosome: {Strand(+ or -): Intervaltree(StartPosition(int), EndPosition(int), {"gene_id": GeneID})}}
	'''

	if ref_introns.split('.')[-1] == 'gz':
		intron_file = gzip.open(ref_introns, 'rt')
	else:
		intron_file = open(ref_introns)

	introns, introns_done = {}, set()
	for line in intron_file:
		chrom, start, end, intron_info, _, strand = line.strip().split('\t')
		if chrom not in introns:
			introns[chrom] = IntervalTree()
		intron_id = '{}_{}_{}_{}'.format(chrom, strand, start, end)
		if intron_id not in introns_done:
			start, end = int(start), int(end)
			if end-start > 20:
				intron_genes = set([i.split('-')[0] for i in intron_info.split(';')[-1].split('|')])
				introns[chrom].add(Interval(start, end, {'strand': strand, 'gene_id': intron_genes}))
				introns_done.add(intron_id)

	intron_file.close()
	return introns


def parse_exons(ref_exons:str) -> tuple:
	'''
	Returns a dict formatted as follows:
	{Chromosome: {Strand(+ or -): Intervaltree(StartPosition(int), EndPosition(int), {"gene_id": GeneID})}}
	'''
	if ref_exons.split('.')[-1] == 'gz':
		exon_file = gzip.open(ref_exons, 'rt')
	else:
		exon_file = open(ref_exons)

	exons, exons_done = {}, set()
	for line in exon_file:
		chrom, start, end, exon_info, _, strand = line.strip().split('\t')
		if chrom not in exons:
			exons[chrom] = IntervalTree()
		exon_id = '{}_{}_{}_{}'.format(chrom, strand, start, end)
		if exon_id not in exons_done:
			start, end = int(start), int(end)
			if end-start > 20:
				exon_genes = set([i.split('-')[0] for i in exon_info.split(';')[-1].split('|')])
				exons[chrom].add(Interval(start, end, {'strand': strand, 'gene_id':exon_genes}))
				exons_done.add(exon_id)

	exon_file.close()
	return exons


def infer_segments(row):
	if len(row['blocks']) == 1:
		return row['blocks']
	
	segs = []
	current_block = row['blocks'][0]
	for blocks_index in range(1, len(row['blocks'])):
		next_block = row['blocks'][blocks_index]
		cigar_index = 2*blocks_index - 1
		gap_type = row['cigar'][cigar_index][0]

		if gap_type == 'N':
			segs.append(current_block)
			current_block = next_block
		elif gap_type in ('I', 'D'):
			current_block = (current_block[0], next_block[1])
		else:
			raise RuntimeError(f"{row['blocks']}\n{row['cigar']}\n{blocks_index}")
	
	segs.append(current_block)
		
	return tuple(segs)


def tree_covers_interval(tree:IntervalTree, interval:Interval) -> bool:
	total_coverage = False
	merged_tree = tree.copy()
	merged_tree.merge_overlaps(strict=False)
	for merged_interval in merged_tree:
		if merged_interval.contains_interval(interval):
			total_coverage = True
	
	return total_coverage

In [4]:
##### Settings #####
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

ref_anno='/datasets2/lariat_mapping/reference_files/human/ref_dir_a/annotation.gtf'
ref_introns='/datasets2/lariat_mapping/reference_files/human/ref_dir_a/introns.bed'
ref_exons='/datasets2/lariat_mapping/reference_files/human/ref_dir_a/exons.bed'
single_end = False
# output_base='/datasets2/lariat_mapping/testing/output/C22-1/'
# output_base='/datasets2/lariat_mapping/testing/output/HEK293T-1/'
# output_base='/datasets2/lariat_mapping/testing/output/C22-1_10m/'
# output_base='/datasets2/lariat_mapping/testing/output/HEK293T-1_10m/'
output_base='/datasets2/lariat_mapping/testing/100k_truncs/C22-1/'
# output_base='/datasets2/lariat_mapping/testing/100k_truncs/HEK293T-1/'
print(output_base)

/datasets2/lariat_mapping/testing/100k_truncs/C22-1/


In [5]:
genes = parse_gene_info(ref_anno)
print(list(genes.items())[0])
introns = parse_introns(ref_introns)
print(list(introns['chr1'])[:3])
exons = parse_exons(ref_exons)
print(list(exons['chr1'])[:3])

('ENSG00000290825.1', ['DDX11L2', 'lncRNA'])
[Interval(15118904, 15120971, {'strand': '-', 'gene_id': {'ENSG00000175147.13'}}), Interval(111977008, 111981620, {'strand': '-', 'gene_id': {'ENSG00000171385.11'}}), Interval(61251369, 61253410, {'strand': '-', 'gene_id': {'ENSG00000237853.6'}})]
[Interval(86569023, 86571408, {'strand': '-', 'gene_id': {'ENSG00000236915.3'}}), Interval(35018512, 35019602, {'strand': '-', 'gene_id': {'ENSG00000163867.17'}}), Interval(155562072, 155562286, {'strand': '+', 'gene_id': {'ENSG00000235919.5'}})]


In [6]:
# Add gene types to introns
for chrom in introns.keys():
	for intron in introns[chrom]:
		gene_info = [genes[gene_id] for gene_id in intron.data['gene_id']]
		gene_types = set([type_ for name, type_ in gene_info])
		intron.data['gene_type'] = gene_types
print(list(introns['chr1'])[:3])

# Add gene types to exons
for chrom in exons.keys():
	for exon in exons[chrom]:
		gene_info = [genes[gene_id] for gene_id in exon.data['gene_id']]
		gene_types = set([type_ for name, type_ in gene_info])
		exon.data['gene_type'] = gene_types
print(list(exons['chr1'])[:3])

[Interval(15118904, 15120971, {'strand': '-', 'gene_id': {'ENSG00000175147.13'}, 'gene_type': {'lncRNA'}}), Interval(111977008, 111981620, {'strand': '-', 'gene_id': {'ENSG00000171385.11'}, 'gene_type': {'protein_coding'}}), Interval(61251369, 61253410, {'strand': '-', 'gene_id': {'ENSG00000237853.6'}, 'gene_type': {'lncRNA'}})]
[Interval(86569023, 86571408, {'strand': '-', 'gene_id': {'ENSG00000236915.3'}, 'gene_type': {'lncRNA'}}), Interval(35018512, 35019602, {'strand': '-', 'gene_id': {'ENSG00000163867.17'}, 'gene_type': {'protein_coding'}}), Interval(155562072, 155562286, {'strand': '+', 'gene_id': {'ENSG00000235919.5'}, 'gene_type': {'lncRNA'}})]


In [7]:
stats_out = {}

In [8]:
linear_reads = []
unmapped_rids = set()
for align in pysam.AlignmentFile(f'{output_base}/mapped_reads.bam', 'rb'):
	if align.is_unmapped:
		unmapped_rids.add(align.query_name)
		continue

	linear_reads.append([
					align.query_name, 
					align.reference_name, 
					# align.reference_start,
					# align.reference_end, 
					align.get_blocks(),
					align.cigartuples,
					# align.get_tags(),
					align.is_read1,
					])

linear_reads = pd.DataFrame(linear_reads, columns=LINEAR_COLUMNS)
linear_reads.cigar = linear_reads.cigar.transform(lambda cigar: tuple((CIGARTUPLE_CODES[op],length) for op, length in cigar))
# linear_reads.tags = linear_reads.tags.transform(lambda tags: {tag: val for tag, val in tags})
if single_end is False:
	linear_reads.read = linear_reads.read.map({True: '1', False: '2'})
	linear_reads['read_id'] = linear_reads.read_id + '/' + linear_reads.read
assert linear_reads.read_id.is_unique
linear_reads = linear_reads.sort_values(['read_id'])
linear_reads

,read_id,chrom,blocks,cigar,read
38756,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/1,chr22,"[(43161160, 43161190), (43162802, 43162922)]","((M, 30), (N, 1612), (M, 120))",1
38757,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/2,chr22,"[(43161094, 43161190), (43162802, 43162856)]","((M, 96), (N, 1612), (M, 54))",2
41412,NGSNJ-086:229:GW200110425th:1:1101:10004:11992/1,chr9,"[(9442096, 9442246)]","((M, 150),)",1
41413,NGSNJ-086:229:GW200110425th:1:1101:10004:11992/2,chr9,"[(9442107, 9442257)]","((M, 150),)",2
41718,NGSNJ-086:229:GW200110425th:1:1101:10004:12085/1,chr14,"[(49862579, 49862729)]","((M, 150),)",1
...,...,...,...,...,...
98697,NGSNJ-086:229:GW200110425th:1:1101:9995:27352/2,chr14,"[(32163953, 32164103)]","((M, 150),)",2
14870,NGSNJ-086:229:GW200110425th:1:1101:9995:4961/1,chr13,"[(48233331, 48233481)]","((M, 150),)",1
14871,NGSNJ-086:229:GW200110425th:1:1101:9995:4961/2,chr13,"[(48233242, 48233392)]","((M, 150),)",2
31980,NGSNJ-086:229:GW200110425th:1:1101:9995:9502/1,chr14,"[(49586704, 49586854)]","((M, 150),)",1


In [9]:
linear_reads['segs'] = linear_reads.apply(infer_segments, axis=1, result_type='reduce')
linear_reads = linear_reads.explode('segs')
linear_reads['seg'] = linear_reads.segs.transform(lambda segs: Interval(*segs))
linear_reads = linear_reads.drop(columns=['blocks', 'read', 'segs', 'cigar'])
linear_reads

,read_id,chrom,seg
38756,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/1,chr22,"(43161160, 43161190, None)"
38756,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/1,chr22,"(43162802, 43162922, None)"
38757,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/2,chr22,"(43161094, 43161190, None)"
38757,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/2,chr22,"(43162802, 43162856, None)"
41412,NGSNJ-086:229:GW200110425th:1:1101:10004:11992/1,chr9,"(9442096, 9442246, None)"
...,...,...,...
14870,NGSNJ-086:229:GW200110425th:1:1101:9995:4961/1,chr13,"(48233331, 48233481, None)"
14871,NGSNJ-086:229:GW200110425th:1:1101:9995:4961/2,chr13,"(48233242, 48233392, None)"
31980,NGSNJ-086:229:GW200110425th:1:1101:9995:9502/1,chr14,"(49586704, 49586854, None)"
31981,NGSNJ-086:229:GW200110425th:1:1101:9995:9502/2,chr14,"(49563422, 49563424, None)"


In [10]:
for chrom in linear_reads.chrom.unique():
	if chrom not in exons.keys() or chrom not in introns.keys():
		print(f'{chrom} not found')
		continue

	chrom_exons = exons[chrom]
	chrom_introns = introns[chrom]
	linear_reads.loc[linear_reads.chrom==chrom, 'exons'] = linear_reads.loc[linear_reads.chrom==chrom, 'seg'].transform(chrom_exons.overlap)
	linear_reads.loc[linear_reads.chrom==chrom, 'introns'] = linear_reads.loc[linear_reads.chrom==chrom, 'seg'].transform(chrom_introns.overlap)

linear_reads.exons = linear_reads.exons.fillna('').transform(set)
linear_reads.introns = linear_reads.introns.fillna('').transform(set)
linear_reads

chrM not found


,read_id,chrom,seg,exons,introns
38756,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/1,chr22,"(43161160, 43161190, None)","{(43161051, 43161190, {'strand': '+', 'gene_id...",{}
38756,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/1,chr22,"(43162802, 43162922, None)","{(43162802, 43163210, {'strand': '+', 'gene_id...",{}
38757,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/2,chr22,"(43161094, 43161190, None)","{(43161051, 43161190, {'strand': '+', 'gene_id...",{}
38757,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/2,chr22,"(43162802, 43162856, None)","{(43162802, 43163210, {'strand': '+', 'gene_id...",{}
41412,NGSNJ-086:229:GW200110425th:1:1101:10004:11992/1,chr9,"(9442096, 9442246, None)","{(9442059, 9442380, {'strand': '+', 'gene_id':...","{(9397482, 9574731, {'strand': '-', 'gene_id':..."
...,...,...,...,...,...
14870,NGSNJ-086:229:GW200110425th:1:1101:9995:4961/1,chr13,"(48233331, 48233481, None)","{(48233202, 48233477, {'strand': '+', 'gene_id...","{(48233477, 48253807, {'strand': '+', 'gene_id..."
14871,NGSNJ-086:229:GW200110425th:1:1101:9995:4961/2,chr13,"(48233242, 48233392, None)","{(48233202, 48233477, {'strand': '+', 'gene_id...","{(48232698, 48253807, {'strand': '+', 'gene_id..."
31980,NGSNJ-086:229:GW200110425th:1:1101:9995:9502/1,chr14,"(49586704, 49586854, None)","{(49586579, 49586878, {'strand': '+', 'gene_id...","{(49586049, 49598399, {'strand': '-', 'gene_id..."
31981,NGSNJ-086:229:GW200110425th:1:1101:9995:9502/2,chr14,"(49563422, 49563424, None)",{},{}


In [11]:
def infer_common_genes(overlap_exons):
	gene_ids = []
	for seg_exons in overlap_exons:
		seg_genes = set()
		for exon in seg_exons:
			seg_genes.update(exon.data['gene_id'])
		gene_ids.append(seg_genes)

	common_genes = set(gid for gid in gene_ids[0] if all(gid in set_genes for set_genes in gene_ids))
	return common_genes

	# filtered_exons = []
	# for seg_exons in overlap_exons:
	# 	filtered_set = set(exon for exon in seg_exons if len(exon.data['gene_id'].intersection(common_genes))>0)
	# 	filtered_exons.append(filtered_set)

	# return filtered_exons

	
linear_reads['common_genes'] = linear_reads.read_id.map(linear_reads.groupby('read_id').exons.agg(infer_common_genes))
linear_reads

,read_id,chrom,seg,exons,introns,common_genes
38756,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/1,chr22,"(43161160, 43161190, None)","{(43161051, 43161190, {'strand': '+', 'gene_id...",{},{ENSG00000100300.18}
38756,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/1,chr22,"(43162802, 43162922, None)","{(43162802, 43163210, {'strand': '+', 'gene_id...",{},{ENSG00000100300.18}
38757,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/2,chr22,"(43161094, 43161190, None)","{(43161051, 43161190, {'strand': '+', 'gene_id...",{},{ENSG00000100300.18}
38757,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/2,chr22,"(43162802, 43162856, None)","{(43162802, 43163210, {'strand': '+', 'gene_id...",{},{ENSG00000100300.18}
41412,NGSNJ-086:229:GW200110425th:1:1101:10004:11992/1,chr9,"(9442096, 9442246, None)","{(9442059, 9442380, {'strand': '+', 'gene_id':...","{(9397482, 9574731, {'strand': '-', 'gene_id':...",{ENSG00000265735.2}
...,...,...,...,...,...,...
14870,NGSNJ-086:229:GW200110425th:1:1101:9995:4961/1,chr13,"(48233331, 48233481, None)","{(48233202, 48233477, {'strand': '+', 'gene_id...","{(48233477, 48253807, {'strand': '+', 'gene_id...",{ENSG00000136156.15}
14871,NGSNJ-086:229:GW200110425th:1:1101:9995:4961/2,chr13,"(48233242, 48233392, None)","{(48233202, 48233477, {'strand': '+', 'gene_id...","{(48232698, 48253807, {'strand': '+', 'gene_id...",{ENSG00000136156.15}
31980,NGSNJ-086:229:GW200110425th:1:1101:9995:9502/1,chr14,"(49586704, 49586854, None)","{(49586579, 49586878, {'strand': '+', 'gene_id...","{(49586049, 49598399, {'strand': '-', 'gene_id...",{ENSG00000276168.1}
31981,NGSNJ-086:229:GW200110425th:1:1101:9995:9502/2,chr14,"(49563422, 49563424, None)",{},{},{}


In [12]:
linear_reads['exons_f'] = linear_reads.apply(lambda row: set(exon for exon in row['exons'] if len(exon.data['gene_id'].intersection(row['common_genes']))>0), axis=1)
print(linear_reads.exons.transform(len).value_counts())
print(linear_reads.exons_f.transform(len).value_counts())

exons
1     42488
0     25870
2     23631
3     19171
4      7616
5      4638
6      3286
7      2071
8      1207
9       877
10      498
11      398
12      331
13      209
14      123
16      119
15       98
17       66
19       52
18       43
20       39
25       32
21       31
26       29
24       22
23       11
22       10
29       10
28        9
39        9
30        6
41        5
27        5
31        5
37        4
32        4
34        4
33        4
36        2
43        2
38        1
35        1
Name: count, dtype: int64
exons_f
1     41399
0     28067
2     23603
3     18355
4      7536
5      4590
6      3302
7      2006
8      1188
9       834
10      496
11      391
12      322
13      207
16      121
14      121
15       97
17       70
19       52
18       41
20       35
25       34
21       29
26       27
24       20
23       13
22       10
29       10
28        9
39        9
30        6
41        5
27        5
31        5
37        4
32        4
34        4
33        4


In [13]:
# def pull_gene_types(row):
# 	out = set()
# 	for exon in row['exons']:
# 		out = out.union(exon.data['gene_type'])
# 	for intron in row['introns']:
# 		out = out.union(intron.data['gene_type'])
		
# 	return out

# linear_reads['gene_types'] = linear_reads.apply(pull_gene_types, axis=1) 
# print(linear_reads.gene_types.value_counts())

In [14]:
def classify_seg(row):
	if any(exon.begin==row['seg'].begin for exon in row['exons']) or any(exon.end==row['seg'].end for exon in row['exons']):
		return 'Exon junction'
		
	if len(row['exons'])==0 and len(row['introns'])==0:
		return 'Inter-genic'

	if len(row['exons'])==0 and len(row['introns'])>0:
		if any(intron.begin==row['seg'].begin for intron in row['introns']) or any(intron.end==row['seg'].end for intron in row['introns']):
			return 'Intron start'
		
		return 'Intronic'

	if len(row['exons'])>0 and len(row['introns'])==0:
		return 'Exonic'

	# We can deduce that there's at least 1 exon and 1 intron at this point
	if tree_covers_interval(IntervalTree(row['exons']), row['seg']) is True:
		return 'Ambiguous'
	if tree_covers_interval(IntervalTree(row['introns']), row['seg']) is True:
		return 'Ambiguous'

	return 'pre-mRNA'

linear_reads['seg_class'] = linear_reads.apply(classify_seg, axis=1)
print(linear_reads.seg_class.value_counts())

seg_class
Exon junction    47155
Ambiguous        34137
Exonic           24829
Intronic         21125
Inter-genic       4733
pre-mRNA          1046
Intron start        12
Name: count, dtype: int64


In [15]:
def classify_read(seg_classes):
	seg_classes_set = set(seg_classes)

	if seg_classes_set == {'Exon junction',}:
		return 'mRNA'

	if len(seg_classes_set) == 1:
		return seg_classes_set.pop()

	if seg_classes_set in (
							{'Exon junction', 'Exonic'},
							{'Exon junction', 'Ambiguous'}
							):
		return 'mRNA'
	
	
	if seg_classes_set in (
						{'Intronic', 'Exonic'},
						{'Intronic', 'Exon junction'},
						{'Intronic', 'Exonic', 'Exon junction'},
						{'pre-mRNA', 'Intronic'},
						{'pre-mRNA', 'Exonic'},
						{'pre-mRNA', 'Exon junction'},
						{'pre-mRNA', 'Intron start'},
						{'pre-mRNA', 'Ambiguous'},
						{'pre-mRNA', 'Exon junction', 'Intronic'},
						{'pre-mRNA', 'Exon junction', 'Exonic'},
						{'pre-mRNA', 'Exonic', 'Intronic'},
						{'pre-mRNA', 'Exon junction', 'Intronic', 'Exonic'},
						):
		return 'pre-mRNA'

	return tuple(seg_classes_set)



linear_reads['read_class'] = linear_reads.read_id.map(linear_reads.groupby('read_id').seg_class.agg(classify_read))
print(linear_reads[['read_id', 'read_class']].read_class.value_counts())
linear_reads

read_class
mRNA                                  47106
Ambiguous                             32475
Exonic                                24380
Intronic                              20640
Inter-genic                            3698
(Ambiguous, Inter-genic)               1847
pre-mRNA                               1281
(Ambiguous, Exonic)                     774
(Ambiguous, Intronic)                   578
(Inter-genic, Intronic)                 185
(Exon junction, Inter-genic)             29
(Inter-genic, Exonic)                    16
Intron start                             11
(Ambiguous, Intronic, Inter-genic)        6
(Ambiguous, Inter-genic, Exonic)          3
(Exon junction, Intron start)             3
(Ambiguous, Exonic, Inter-genic)          3
(Inter-genic, pre-mRNA)                   2
Name: count, dtype: int64


,read_id,chrom,seg,exons,introns,common_genes,exons_f,seg_class,read_class
38756,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/1,chr22,"(43161160, 43161190, None)","{(43161051, 43161190, {'strand': '+', 'gene_id...",{},{ENSG00000100300.18},"{(43161051, 43161190, {'strand': '+', 'gene_id...",Exon junction,mRNA
38756,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/1,chr22,"(43162802, 43162922, None)","{(43162802, 43163210, {'strand': '+', 'gene_id...",{},{ENSG00000100300.18},"{(43162802, 43163210, {'strand': '+', 'gene_id...",Exon junction,mRNA
38757,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/2,chr22,"(43161094, 43161190, None)","{(43161051, 43161190, {'strand': '+', 'gene_id...",{},{ENSG00000100300.18},"{(43161051, 43161190, {'strand': '+', 'gene_id...",Exon junction,mRNA
38757,NGSNJ-086:229:GW200110425th:1:1101:10004:11303/2,chr22,"(43162802, 43162856, None)","{(43162802, 43163210, {'strand': '+', 'gene_id...",{},{ENSG00000100300.18},"{(43162802, 43163210, {'strand': '+', 'gene_id...",Exon junction,mRNA
41412,NGSNJ-086:229:GW200110425th:1:1101:10004:11992/1,chr9,"(9442096, 9442246, None)","{(9442059, 9442380, {'strand': '+', 'gene_id':...","{(9397482, 9574731, {'strand': '-', 'gene_id':...",{ENSG00000265735.2},"{(9442059, 9442380, {'strand': '+', 'gene_id':...",Ambiguous,Ambiguous
...,...,...,...,...,...,...,...,...,...
14870,NGSNJ-086:229:GW200110425th:1:1101:9995:4961/1,chr13,"(48233331, 48233481, None)","{(48233202, 48233477, {'strand': '+', 'gene_id...","{(48233477, 48253807, {'strand': '+', 'gene_id...",{ENSG00000136156.15},"{(48233202, 48233477, {'strand': '+', 'gene_id...",Ambiguous,Ambiguous
14871,NGSNJ-086:229:GW200110425th:1:1101:9995:4961/2,chr13,"(48233242, 48233392, None)","{(48233202, 48233477, {'strand': '+', 'gene_id...","{(48232698, 48253807, {'strand': '+', 'gene_id...",{ENSG00000136156.15},"{(48233202, 48233477, {'strand': '+', 'gene_id...",Ambiguous,Ambiguous
31980,NGSNJ-086:229:GW200110425th:1:1101:9995:9502/1,chr14,"(49586704, 49586854, None)","{(49586579, 49586878, {'strand': '+', 'gene_id...","{(49586049, 49598399, {'strand': '-', 'gene_id...",{ENSG00000276168.1},"{(49586579, 49586878, {'strand': '+', 'gene_id...",Ambiguous,Ambiguous
31981,NGSNJ-086:229:GW200110425th:1:1101:9995:9502/2,chr14,"(49563422, 49563424, None)",{},{},{},{},Inter-genic,"(Ambiguous, Inter-genic)"


In [16]:
def collapse_mate_classes(classes):
	classes_set = set(classes)

	if len(classes_set)==1:
		return classes_set.pop()

	if classes_set in (
					{'mRNA', 'Exonic'},
					{'mRNA', 'Ambiguous'},
					):
		return 'mRNA'

	if classes_set in (
					{'pre-mRNA', 'Exonic'},
					{'pre-mRNA', 'Intronic'},
					{'pre-mRNA', 'mRNA'},
					{'pre-mRNA', 'Ambiguous'},
					{'mRNA', 'Intronic'},
					{'Exonic', 'Intronic'},
					):
		return 'pre-mRNA'

	return tuple(classes_set)



linear_reads.read_class = linear_reads.read_class.transform(lambda read_class: 'Ambiguous' if isinstance(read_class, tuple) else read_class)

if single_end is False:
	linear_reads.read_id = linear_reads.read_id.str.slice(0, -2)
	linear_reads['read_class'] = linear_reads.read_id.map(linear_reads.groupby('read_id').read_class.agg(collapse_mate_classes))

print(linear_reads.read_class.value_counts())

read_class
mRNA                         49686
Ambiguous                    33642
Exonic                       21982
Intronic                     19819
Inter-genic                   3357
pre-mRNA                      2380
(Ambiguous, Intronic)          775
(Ambiguous, Exonic)            662
(Inter-genic, Intronic)        581
(Inter-genic, Exonic)           64
(Ambiguous, Inter-genic)        36
(Inter-genic, mRNA)             27
(Intronic, Intron start)        18
(Inter-genic, pre-mRNA)          4
(pre-mRNA, Intron start)         2
(Ambiguous, Intron start)        2
Name: count, dtype: int64


In [17]:
linear_reads.read_class = linear_reads.read_class.transform(lambda read_class: 'Ambiguous' if isinstance(read_class, tuple) else read_class)
linear_reads_sum = linear_reads[['read_id', 'read_class']].drop_duplicates(ignore_index=False)
print(linear_reads_sum.read_class.value_counts())

read_class
Ambiguous      17013
mRNA           12281
Exonic         10980
Intronic        9815
Inter-genic     1644
pre-mRNA         939
Name: count, dtype: int64


In [29]:
stats_out.update(linear_reads_sum.read_class.value_counts().to_dict())
stats_out['Linear reads'] = linear_reads.read_id.nunique()
stats_out['Spliced Linear reads'] = linear_reads.segs.transform(len).gt(1).sum()
stats_out['Unspliced Linear reads'] = linear_reads.segs.transform(len).eq(1).sum()
stats_out

{'Ambiguous': 17013,
 'mRNA': 12281,
 'Exonic': 10980,
 'Intronic': 9815,
 'Inter-genic': 1644,
 'pre-mRNA': 939}

In [18]:
lariat_rids = pd.read_csv(f'{output_base}lariat_reads.tsv', sep='\t').read_id
lariat_rids = set(lariat_rids)
print(len(lariat_rids))
lariat_rids

7


{'NGSNJ-086:229:GW200110425th:1:1101:15582:16830',
 'NGSNJ-086:229:GW200110425th:1:1101:15700:19695',
 'NGSNJ-086:229:GW200110425th:1:1101:2166:13448',
 'NGSNJ-086:229:GW200110425th:1:1101:24551:5713',
 'NGSNJ-086:229:GW200110425th:1:1101:30291:23797',
 'NGSNJ-086:229:GW200110425th:1:1101:5394:15374',
 'NGSNJ-086:229:GW200110425th:1:1101:8314:28354'}

In [19]:
circular_rids = pd.read_csv(f'{output_base}circularized_intron_reads.tsv', sep='\t').read_id
circular_rids = set(circular_rids)
print(len(circular_rids))
circular_rids

0


set()

In [20]:
temp_switch_rids = pd.read_csv(f'{output_base}template_switching_alignments.tsv', sep='\t').read_id
temp_switch_rids = set(temp_switch_rids)
print(len(temp_switch_rids))
temp_switch_rids

1856


{'NGSNJ-086:229:GW200110425th:1:1101:21875:25629',
 'NGSNJ-086:229:GW200110425th:1:1101:15528:21151',
 'NGSNJ-086:229:GW200110425th:1:1101:6912:20979',
 'NGSNJ-086:229:GW200110425th:1:1101:18168:20431',
 'NGSNJ-086:229:GW200110425th:1:1101:28926:27915',
 'NGSNJ-086:229:GW200110425th:1:1101:21712:7435',
 'NGSNJ-086:229:GW200110425th:1:1101:16613:11569',
 'NGSNJ-086:229:GW200110425th:1:1101:13069:25473',
 'NGSNJ-086:229:GW200110425th:1:1101:19822:20134',
 'NGSNJ-086:229:GW200110425th:1:1101:27462:21527',
 'NGSNJ-086:229:GW200110425th:1:1101:18954:23015',
 'NGSNJ-086:229:GW200110425th:1:1101:27697:22842',
 'NGSNJ-086:229:GW200110425th:1:1101:25861:14465',
 'NGSNJ-086:229:GW200110425th:1:1101:14353:10692',
 'NGSNJ-086:229:GW200110425th:1:1101:2917:14653',
 'NGSNJ-086:229:GW200110425th:1:1101:9679:8954',
 'NGSNJ-086:229:GW200110425th:1:1101:27208:4679',
 'NGSNJ-086:229:GW200110425th:1:1101:4779:27712',
 'NGSNJ-086:229:GW200110425th:1:1101:18123:15342',
 'NGSNJ-086:229:GW200110425th:1:1101:3

In [21]:
assert len(lariat_rids.intersection(circular_rids)) == 0, lariat_rids.intersection(circular_rids)
assert len(lariat_rids.intersection(temp_switch_rids)) == 0, lariat_rids.intersection(temp_switch_rids)
assert len(temp_switch_rids.intersection(circular_rids)) == 0, temp_switch_rids.intersection(circular_rids)

In [22]:
lariat_failed = pd.read_csv(f'{output_base}failed_lariat_alignments.tsv', sep='\t')
lariat_failed = lariat_failed[['read_id', 'filter_failed']]
repeat_rids = lariat_failed.loc[lariat_failed.filter_failed.isin(('in_repeat', 'ubiquitin_gene'))]
print(len(repeat_rids))
lariat_failed

0


,read_id,filter_failed
0,NGSNJ-086:229:GW200110425th:1:1101:24551:5713,not_chosen
1,NGSNJ-086:229:GW200110425th:1:1101:5394:15374,not_chosen
2,NGSNJ-086:229:GW200110425th:1:1101:8314:28354,not_chosen


In [23]:
trim_failed = pd.read_csv(f'{output_base}failed_trimmed_alignments.tsv', sep='\t')
trim_failed.read_id = trim_failed.read_id.transform(lambda rid: rid[:-4].split('/')[0])
trim_failed = trim_failed[['read_id', 'filter_failed']].drop_duplicates(ignore_index=True)
trim_failed

,read_id,filter_failed
0,NGSNJ-086:229:GW200110425th:1:1101:18765:2801,overlaps_intron
1,NGSNJ-086:229:GW200110425th:1:1101:1136:13448,overlaps_intron
2,NGSNJ-086:229:GW200110425th:1:1101:31702:22827,overlaps_intron
3,NGSNJ-086:229:GW200110425th:1:1101:18665:1532,overlaps_intron
4,NGSNJ-086:229:GW200110425th:1:1101:21992:19194,overlaps_intron
...,...,...
4747,NGSNJ-086:229:GW200110425th:1:1101:7618:15812,fivep_intron_match
4748,NGSNJ-086:229:GW200110425th:1:1101:19208:19445,fivep_intron_match
4749,NGSNJ-086:229:GW200110425th:1:1101:13295:19069,fivep_intron_match
4750,NGSNJ-086:229:GW200110425th:1:1101:21938:5791,fivep_intron_match


In [24]:
fivep_passed_rids = pd.read_csv(f'{output_base}fivep_info_table.tsv', sep='\t').read_id
fivep_passed_rids = set(fivep_passed_rids.transform(lambda rid: rid[:-4].split('/')[0]))
print(len(fivep_passed_rids))
fivep_passed_rids

6862


{'NGSNJ-086:229:GW200110425th:1:1101:22363:27164',
 'NGSNJ-086:229:GW200110425th:1:1101:1108:1188',
 'NGSNJ-086:229:GW200110425th:1:1101:11288:9173',
 'NGSNJ-086:229:GW200110425th:1:1101:18168:20431',
 'NGSNJ-086:229:GW200110425th:1:1101:21441:29105',
 'NGSNJ-086:229:GW200110425th:1:1101:29396:28729',
 'NGSNJ-086:229:GW200110425th:1:1101:17960:18724',
 'NGSNJ-086:229:GW200110425th:1:1101:19822:20134',
 'NGSNJ-086:229:GW200110425th:1:1101:23619:7075',
 'NGSNJ-086:229:GW200110425th:1:1101:22598:1986',
 'NGSNJ-086:229:GW200110425th:1:1101:30617:19069',
 'NGSNJ-086:229:GW200110425th:1:1101:2365:6496',
 'NGSNJ-086:229:GW200110425th:1:1101:26223:14653',
 'NGSNJ-086:229:GW200110425th:1:1101:27344:11177',
 'NGSNJ-086:229:GW200110425th:1:1101:6072:16861',
 'NGSNJ-086:229:GW200110425th:1:1101:2917:14653',
 'NGSNJ-086:229:GW200110425th:1:1101:29297:10019',
 'NGSNJ-086:229:GW200110425th:1:1101:25156:27085',
 'NGSNJ-086:229:GW200110425th:1:1101:18123:15342',
 'NGSNJ-086:229:GW200110425th:1:1101:317

In [25]:
read_class = []
for read_id in unmapped_rids:
	if read_id in lariat_rids:
		read_class.append((read_id, 'Lariat'))
	elif read_id in repeat_rids:
		read_class.append((read_id, 'Repeat region'))
	elif read_id in circular_rids:
		read_class.append((read_id, 'Circularized intron'))
	elif read_id in temp_switch_rids:
		read_class.append((read_id, 'Template-switching'))
	elif read_id in fivep_passed_rids:
		read_class.append((read_id, "Has 5'ss alignment"))
	else:
		read_class.append((read_id, 'Unmapped'))

read_class = pd.DataFrame(read_class, columns=('read_id', 'class_'))
print(read_class.class_.value_counts())
assert read_class.read_id.is_unique
read_class

class_
Unmapped              40466
Has 5'ss alignment     4999
Template-switching     1856
Lariat                    7
Name: count, dtype: int64


,read_id,class_
0,NGSNJ-086:229:GW200110425th:1:1101:12038:7654,Unmapped
1,NGSNJ-086:229:GW200110425th:1:1101:1127:18662,Unmapped
2,NGSNJ-086:229:GW200110425th:1:1101:9616:28291,Unmapped
3,NGSNJ-086:229:GW200110425th:1:1101:27688:8140,Unmapped
4,NGSNJ-086:229:GW200110425th:1:1101:14100:15671,Unmapped
...,...,...
47323,NGSNJ-086:229:GW200110425th:1:1101:23005:14215,Unmapped
47324,NGSNJ-086:229:GW200110425th:1:1101:30852:29058,Unmapped
47325,NGSNJ-086:229:GW200110425th:1:1101:9471:10222,Unmapped
47326,NGSNJ-086:229:GW200110425th:1:1101:24903:7294,Unmapped


In [30]:
read_class_sum = read_class.class_.value_counts().to_dict()
print(read_class_sum)

{'Unmapped': 40466, "Has 5'ss alignment": 4999, 'Template-switching': 1856, 'Lariat': 7}


In [32]:
stats_out.update(read_class_sum)
stats_out['Nonlinear reads'] = len(read_class)
stats_out

{'Ambiguous': 17013,
 'mRNA': 12281,
 'Exonic': 10980,
 'Intronic': 9815,
 'Inter-genic': 1644,
 'pre-mRNA': 939,
 'Unmapped': 40466,
 "Has 5'ss alignment": 4999,
 'Template-switching': 1856,
 'Lariat': 7}

##### Code snippets

In [27]:
# plot = (p9.ggplot(, p9.aes(x="", y="")) +
# 		# p9.geom_() +
# 		# p9.geom_() +
# 		# p9.scale_x_() +
# 		# p9.scale_y_() +
# 		# p9.scale_color_brewer(type='', palette=1, direction=1) +
# 		# p9.scale_fill_brewer(type='', palette=1, direction=1) +
# 		p9.labs(title="") +
# 		p9.theme()
# )
# plot.show()
# # plot.save(PLOTS_DIR + '/.png', dpi=500)

##### Discarded code

In [28]:
%%script false --no-raise-error
def add_exons(row):
	overlap_exons = exons[row['chrom']].overlap(row['seg'].begin, row['seg'].end)
	# if any('protein_coding' in exon.data['gene_type'] for exon in overlap_exons):
	overlap_exons = {exon for exon in overlap_exons if 'protein_coding' in exon.data['gene_type']}

	return IntervalTree(overlap_exons)


def add_introns(row):
	if row['chrom'] not in introns.keys():
		return IntervalTree()

	overlap_introns = introns[row['chrom']].overlap(row['seg'].begin, row['seg'].end)
	# if any('protein_coding' in intron.data['gene_type'] for intron in overlap_introns):
	overlap_introns = {intron for intron in overlap_introns if 'protein_coding' in intron.data['gene_type']}

	return IntervalTree(overlap_introns)



# # linear_reads['indels'] = linear_reads.cigar.transform(lambda cigar: sum(length for op, length in cigar if op in ('I', 'D')))
# # linear_reads['mismatches'] = linear_reads.tags.transform(lambda tags: tags['XM'])
# # linear_reads['edit_dist'] = linear_reads.tags.transform(lambda tags: tags['NM'])
# # print(linear_reads.indels.value_counts())
# # print(linear_reads.mismatches.value_counts())
# # print(linear_reads.edit_dist.value_counts())
# linear_reads['spliced'] = linear_reads.n_segs>1
# print(linear_reads.spliced.value_counts())
# linear_reads